In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import trange


import flygym.common
import flygym.mujoco
import flygym.mujoco.preprogrammed

from flygym.mujoco import Parameters


from nmf_grooming import (
    NeuromechflyGrooming,
    all_groom_dofs,
    load_grooming_data,
    plot_state_and_contacts,
)
from scipy.signal import find_peaks, medfilt
from IPython.display import Video

from flygym.mujoco.examples.rule_based_controller import PreprogrammedSteps

In [ ]:
run_time = 1

sim_params = flygym.mujoco.Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.2,
    enable_adhesion=True,
    draw_adhesion=True,
)

actuated_joints = flygym.mujoco.preprogrammed.all_leg_dofs

In [ ]:
target_num_steps = int(run_time / sim_params.timestep) # 10'000


In [ ]:
nmf = flygym.mujoco.NeuroMechFly(Parameters(
    enable_adhesion=True, 
    draw_adhesion=True,
    render_camera="Animat/camera_left"))

preprogrammed_steps = PreprogrammedSteps()

swing_periods = preprogrammed_steps.swing_period

legs = preprogrammed_steps.legs

actuated_joints = preprogrammed_steps.dofs_per_leg

standing_action = []

for leg in legs:
    if leg.endswith("M"):
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, swing_periods[leg][1]))
    else:
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, 0.0))

standing_action = {'joints': standing_action, "adhesion": np.zeros(len(legs))}

for i in range(int(0.2//nmf.timestep)):
    nmf.step(standing_action)



In [ ]:
foreleg_ids = np.zeros(target_num_steps)
middle_stance_ids = np.linspace(swing_periods["RM"][1], -1/4*np.pi, target_num_steps)
hind_swings_ids = np.zeros(target_num_steps)

adhesion_action = np.array([1.0 if leg.endswith("F") else 0.0 for leg in legs])

all_joint_angles = []

for i in range(target_num_steps):
    joint_angles = []
    for leg in legs:
        if leg.endswith("H"):
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, hind_swings_ids[i]))
        elif leg.endswith("M"):
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, middle_stance_ids[i]))
        else:
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, foreleg_ids[i]))
    all_joint_angles.append(joint_angles.copy())

    action = {'joints': np.array(joint_angles), "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()

last_joint_angles = all_joint_angles[-1]

In [ ]:
all_joint_angles = np.ones((target_num_steps, len(all_joint_angles[0])))

for i in range(len(all_joint_angles[0])):
    if i == 5: # LF tibia
        all_joint_angles[:,i] = np.linspace(last_joint_angles[5], 1/3*np.pi, target_num_steps)
    elif i == 6 : # LF tarsus
        all_joint_angles[:,i] = np.linspace(last_joint_angles[6], 0, target_num_steps)
    elif i == 12: # LM tibia
        all_joint_angles[:,i] = np.linspace(last_joint_angles[12], 1/3*np.pi, target_num_steps)
    elif i == 13: # LM tarsus
        all_joint_angles[:,i] = np.linspace(last_joint_angles[13], 0, target_num_steps)
    elif i == 26: # RF tibia
        all_joint_angles[:,i] = np.linspace(last_joint_angles[26], 1/3*np.pi, target_num_steps)
    elif i == 27: # RF tarsus
        all_joint_angles[:,i] = np.linspace(last_joint_angles[27], 0, target_num_steps)
    elif i == 33: # RM tibia
        all_joint_angles[:,i] = np.linspace(last_joint_angles[33], 1/3*np.pi, target_num_steps)
    elif i == 34: # RM tarsus
        all_joint_angles[:,i] = np.linspace(last_joint_angles[34], 0, target_num_steps)
    else:
        all_joint_angles[:,i] = np.linspace(last_joint_angles[i], last_joint_angles[i], target_num_steps)



adhesion_action = np.array([0.0 if leg.endswith("H") else 1.0 for leg in legs])

for i in range(target_num_steps):
    joint_angles = all_joint_angles[i,:]
    action = {'joints': joint_angles, "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()
nmf.save_video("./outputs/b.mp4")

In [ ]:
Video("./outputs/b.mp4")

#### Load grooming position of frontleg

In [ ]:
grooming_module_path = Path("./data/grooming_modules_provided_slow.pkl")
with open(grooming_module_path, "rb") as f:
    grooming_modules = pickle.load(f)

timestep_groom = grooming_modules["timestep"]
target_num_steps_groom = int(1.0 / timestep_groom) # 10'000
target_joint_angles_front = grooming_modules["foreleg"][:, :target_num_steps]

nbre_step = len(grooming_modules['foreleg'][0])-1
replay_steps = [nbre_step,2*nbre_step,3*nbre_step,4*nbre_step,5*nbre_step, 6*nbre_step,
                7*nbre_step,8*nbre_step,9*nbre_step,10*nbre_step,11*nbre_step,12*nbre_step,
                13*nbre_step,14*nbre_step,15*nbre_step,16*nbre_step,17*nbre_step,18*nbre_step]




In [ ]:
# all_joint_angles.shape() = (10000, 42)
# my all_joint_angles_groom_shape() = (558, 21)

# need to map the 21 joints to the 42 joints
target_joint_angles_front = target_joint_angles_front.T # (558, 21)
target_joint_angles_front = np.tile(target_joint_angles_front, (18, 1)) # (10044, 21)

joint_LHCoxa = target_joint_angles_front[:,3]
joint_LHCoxa_roll = target_joint_angles_front[:,4]
joint_LHCoxa_yaw = target_joint_angles_front[:,5]
joint_LHFemur = target_joint_angles_front[:,6]
joint_LHFemur_roll = target_joint_angles_front[:,7]
joint_LHTibia = target_joint_angles_front[:,8]
joint_LHTarsus = target_joint_angles_front[:,9]
joint_RHCoxa = target_joint_angles_front[:,10]
joint_RHCoxa_roll = target_joint_angles_front[:,11]
joint_RHCoxa_yaw = target_joint_angles_front[:,12]
joint_RHFemur = target_joint_angles_front[:,13]
joint_RHFemur_roll = target_joint_angles_front[:,14]
joint_RHTibia = target_joint_angles_front[:,15]
joint_RHTarsus = target_joint_angles_front[:,16]

all_joint_angles = np.ones((target_num_steps, len(all_joint_angles[0]))) # 10000, 42
print(all_joint_angles.shape)
all_joint_angles[:,14] = joint_LHCoxa[:target_num_steps] # jusqu'à 10'000
all_joint_angles[:,15] = joint_LHCoxa_roll[:target_num_steps]
all_joint_angles[:,16] = joint_LHCoxa_yaw[:target_num_steps]
all_joint_angles[:,17] = joint_LHFemur[:target_num_steps]
all_joint_angles[:,18] = joint_LHFemur_roll[:target_num_steps]
all_joint_angles[:,19] = joint_LHTibia[:target_num_steps]
all_joint_angles[:,20] = joint_LHTarsus[:target_num_steps]
all_joint_angles[:,35] = joint_RHCoxa[:target_num_steps]
all_joint_angles[:,36] = joint_RHCoxa_roll[:target_num_steps]
all_joint_angles[:,37] = joint_RHCoxa_yaw[:target_num_steps]
all_joint_angles[:,38] = joint_RHFemur[:target_num_steps]
all_joint_angles[:,39] = joint_RHFemur_roll[:target_num_steps]
all_joint_angles[:,40] = joint_RHTibia[:target_num_steps]
all_joint_angles[:,41] = joint_RHTarsus[:target_num_steps]


k = 0
for i in trange(target_num_steps):
    joint_angles = all_joint_angles[i,:]
    
    action = {"joints": joint_angles, "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()
    if i in replay_steps:
        k = 0
    else:
        k += 1

nmf.save_video("./outputs/b.mp4")